#Análise Exploratória dos Dados

In [61]:
import pandas as pd
import gdown
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


##Leitura e limpeza dos dados

###Importação dos dados

In [62]:
file_id = '1t_3xs96ce44UiVYJ8KqG3hOxRWllXZYy'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'data.csv'
gdown.download(url, output, quiet=False)
df = pd.read_csv('data.csv', sep=',')
# df.head()

Downloading...
From: https://drive.google.com/uc?id=1t_3xs96ce44UiVYJ8KqG3hOxRWllXZYy
To: /content/data.csv
100%|██████████| 303k/303k [00:00<00:00, 58.0MB/s]


###Exclusão de colunas desnecessárias

In [63]:
df = df.drop('Unnamed: 0', axis=1)
# df.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


###Transformando a coluna runtime em apenas números inteiros

In [64]:
df['Runtime'] = df['Runtime'].str.replace('min', '').astype(int)
# df.head()

###Separação das varíaveis categóricas e numéricas

In [65]:
categorical_cols = [
    'Series_Title', 'Released_Year', 'Certificate', 'Genre', 'Overview',
    'Director', 'Star1', 'Star2', 'Star3', 'Star4'
]

numerical_cols = [
    'IMDB_Rating', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross'
]

###Verificando e tratando quantidade de nulos

In [66]:
df.isnull().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

Vemos que temos alguns dados sem algumas informações, como Faturamento, Classificação Etária e Média ponderada de todas as críticas. Vamos verificar quais tipos de filmes estão mais propensos a ter valores nulos.

In [67]:
#Transformar ano de lançamento em númerico
#   Apresenta erro, pois o filme apollo 13 está com seu ano de lançamento como
#   'PG', após pesquisa foi verificado que seu ano de lançamento é 1995, vamos
#   substituir isso no dataset:
df.loc[df['Series_Title'] == 'Apollo 13', 'Released_Year'] = 1995
df[df['Series_Title'] == 'Apollo 13']
df['Released_Year'] = df['Released_Year'].astype('int64')

In [68]:
#Analisando se podemos excluir os valores nulos sem perder dados importantes

# pegando os dados antes dos anos 2000
df_before_2000 = df[df['Released_Year'] <= 2000]
df_before_2000_gross_null = df_before_2000[df_before_2000['Gross'].isnull()]
df_before_2000_gross_not_null = df_before_2000[~df_before_2000['Gross'].isnull()]

df_before_2000_meta_null = df_before_2000[df_before_2000['Meta_score'].isnull()]
df_before_2000_meta_not_null = df_before_2000[~df_before_2000['Meta_score'].isnull()]

df_before_2000_certificate_null = df_before_2000[df_before_2000['Certificate'].isnull()]
df_before_2000_certificate_not_null = df_before_2000[~df_before_2000['Certificate'].isnull()]

# pegando os dados depois dos anos 2000
df_after_2000 = df[df['Released_Year'] >= 2000]
df_after_2000_gross_null = df_after_2000[df_after_2000['Gross'].isnull()]
df_after_2000_gross_not_null = df_after_2000[~df_after_2000['Gross'].isnull()]

df_after_2000_meta_null = df_after_2000[df_after_2000['Meta_score'].isnull()]
df_after_2000_meta_not_null = df_after_2000[~df_after_2000['Meta_score'].isnull()]

df_after_2000_certificate_null = df_after_2000[df_after_2000['Certificate'].isnull()]
df_after_2000_certificate_not_null = df_after_2000[~df_after_2000['Certificate'].isnull()]

print("Antes dos anos 2000:")
print("- Filmes com faturamento nulo:", len(df_before_2000_gross_null))
print("- Filmes com faturamento não nulo:", len(df_before_2000_gross_not_null))
print("- Filmes com meta_score nulo:", len(df_before_2000_meta_null))
print("- Filmes com meta_score não nulo:", len(df_before_2000_meta_not_null))
print("- Filmes com certificate nulo:", len(df_before_2000_certificate_null))
print("- Filmes com certificate não nulo:", len(df_before_2000_certificate_not_null))

print("\n")

print("Depois dos anos 2000:")
print("- Filmes com faturamento nulo:", len(df_after_2000_gross_null))
print("- Filmes com faturamento não nulo:", len(df_after_2000_gross_not_null))
print("- Filmes com média ponderada das críticas nulo:", len(df_after_2000_meta_null))
print("- Filmes com média ponderada das críticas não nulo:", len(df_after_2000_meta_not_null))
print("- Filmes com classificação etária nulo:", len(df_after_2000_certificate_null))
print("- Filmes com classificação etária não nulo:", len(df_after_2000_certificate_not_null))

Antes dos anos 2000:
- Filmes com faturamento nulo: 117
- Filmes com faturamento não nulo: 416
- Filmes com meta_score nulo: 91
- Filmes com meta_score não nulo: 442
- Filmes com certificate nulo: 66
- Filmes com certificate não nulo: 467


Depois dos anos 2000:
- Filmes com faturamento nulo: 55
- Filmes com faturamento não nulo: 430
- Filmes com média ponderada das críticas nulo: 67
- Filmes com média ponderada das críticas não nulo: 418
- Filmes com classificação etária nulo: 37
- Filmes com classificação etária não nulo: 448


Filmes anteriores a 2000 têm mais valores nulos, que podem ocorrer devido à menor disponibilidade de dados e mudanças nos padrões de coleta. Ao analisar a matriz de correlação, podemos retirar os valores nulos correspondentes ao faturamento (Gross), a média ponderada das críticas (meta_score) e a classificação etária (Certificate), para assim termos uma machine learning mais consistente.


In [69]:
df = df.dropna(subset=['Gross', 'Meta_score', 'Certificate'])
df.isnull().sum()


Series_Title     0
Released_Year    0
Certificate      0
Runtime          0
Genre            0
IMDB_Rating      0
Overview         0
Meta_score       0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
No_of_Votes      0
Gross            0
dtype: int64

### Converter colunas numéricas com vírgulas para tipo Float

In [ ]:
for col in numerical_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '', regex=True).astype(float)

###Buscando o nível de correlação entre as variáveis numéricas

In [70]:
# Criar matriz de correlação
corr_matrix = df[numerical_cols].corr()

fig = px.imshow(corr_matrix, text_auto=True)

fig.update_layout(
    title="Heatmap das correlações",
    xaxis_title="Variáveis",
    yaxis_title="Variáveis",
    width=600,
    height=500
)

fig.show()


##Análise dos dados

####Análises dos Diretores

In [71]:
director_gross = df.groupby('Director')['Gross'].sum().sort_values(ascending=False)

top_10_directors = director_gross.head(10)

fig = px.bar(top_10_directors, x=top_10_directors.index, y='Gross',
             title='Diretores com maior faturamento',
             labels={'Director': 'Diretor', 'Gross': 'Faturamento Total (USD)'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [72]:
avg_gross_director = df.groupby('Director')['Gross'].mean()
avg_gross_director = avg_gross_director.sort_values(ascending=False)
top_directors = avg_gross_director.head(10)

fig = px.bar(top_directors, x=top_directors.index, y='Gross',
             title = "Top 10 diretores com maior faturamento médio",
             labels={'Director': 'Diretor', 'Gross': 'Faturamento médio (USD)'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [73]:
director_movies = df.groupby('Director')['Series_Title'].count().sort_values(ascending=False)

top_directors_movies = director_movies.head(10)

fig = px.bar(top_directors_movies, x=top_directors_movies.index, y='Series_Title',
             title = "Top 10 diretores com mais filmes",
             labels={'Director': 'Diretor', 'Series_Title': 'Quantidade de filmes'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [74]:
director_rating = df.groupby('Director')['IMDB_Rating'].max()

director_rating = director_rating.sort_values(ascending=False)
top_directors_rating = director_rating.head(10)

fig = px.bar(top_directors_rating, x=top_directors_rating.index, y='IMDB_Rating',
             title = "Top 10 diretores com maior nota no IMDB",
             labels={'Director': 'Diretor', 'IMDB_Rating': 'Nota do IMDB'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [75]:
director_rating = df.groupby('Director')['IMDB_Rating'].mean()

director_rating = director_rating.sort_values(ascending=False)
top_directors_rating = director_rating.head(10)

fig = px.bar(top_directors_rating, x=top_directors_rating.index, y='IMDB_Rating',
             title = "Top 10 diretores com maior nota média no IMDB",
             labels={'Director': 'Diretor', 'IMDB_Rating': 'Nota do IMDB'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [76]:
# Criando um dicionário para armazenar os gêneros de cada diretor
director_genres = {}
for director in top_10_directors.index:
    movies = df[df['Director'] == director]['Genre']
    genres = set()
    for movie in movies:
        genres.update(movie.split(','))
    director_genres[director] = genres

# Contagem o número de filmes de cada gênero para cada diretor
director_genre_counts = {}
for director, genres in director_genres.items():
    genre_counts = {}
    for genre in genres:
        genre_counts[genre] = 0
    for movie in df[df['Director'] == director]['Genre']:
        for genre in movie.split(','):
            genre_counts[genre] += 1
    director_genre_counts[director] = genre_counts

# Criar um dataframe com os gêneros e contagens para cada diretor
df_director_genres = pd.DataFrame(director_genre_counts).transpose()

fig = px.bar(df_director_genres,
             title='Gêneros mais produzidos pelos Top 10 diretores',
             labels={'index': 'Diretor', 'value': 'Quantidade de filmes'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

####Análises das Estrelas

In [77]:
#Juntando as estrelas dos filmes e os seus respectivos faturamentos
stars = pd.concat([df[['Star1', 'Gross']].rename(columns={'Star1': 'Star'}),
                   df[['Star2', 'Gross']].rename(columns={'Star2': 'Star'}),
                   df[['Star3', 'Gross']].rename(columns={'Star3': 'Star'}),
                   df[['Star4', 'Gross']].rename(columns={'Star4': 'Star'})])

star_gross = stars.groupby('Star')['Gross'].sum().sort_values(ascending=False)
top_stars = star_gross.head(10)

fig = px.bar(top_stars, x=top_stars.index, y='Gross',
             title='Estrelas com maior faturamento',
             labels={'Star1': 'Estrela', 'Gross': 'Faturamento Total (USD)'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [78]:
#Juntando as estrelas dos filmes e as suas respectivas notas no IMDB
stars = pd.concat([df[['Star1', 'IMDB_Rating']].rename(columns={'Star1': 'Star'}),
                   df[['Star2', 'IMDB_Rating']].rename(columns={'Star2': 'Star'}),
                   df[['Star3', 'IMDB_Rating']].rename(columns={'Star3': 'Star'}),
                   df[['Star4', 'IMDB_Rating']].rename(columns={'Star4': 'Star'})])

star_rating = stars.groupby('Star')['IMDB_Rating'].max().sort_values(ascending=False)
top_stars_rating = star_rating.head(10)

fig = px.bar(top_stars_rating, x=top_stars_rating.index, y='IMDB_Rating',
             title='Estrelas com maior média de nota no IMDB',
             labels={'Star1': 'Estrela', 'IMDB_Rating': 'Média de nota no IMDB'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [79]:
star_rating = stars.groupby('Star')['IMDB_Rating'].mean().sort_values(ascending=False)
top_stars_rating = star_rating.head(10)

fig = px.bar(top_stars_rating, x=top_stars_rating.index, y='IMDB_Rating',
             title='Estrelas com maior média de nota no IMDB',
             labels={'Star1': 'Estrela', 'IMDB_Rating': 'Média de nota no IMDB'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()


In [80]:
stars = pd.concat([df[['Star1', 'Genre']].rename(columns={'Star1': 'Star'}),
                   df[['Star2', 'Genre']].rename(columns={'Star2': 'Star'}),
                   df[['Star3', 'Genre']].rename(columns={'Star3': 'Star'}),
                   df[['Star4', 'Genre']].rename(columns={'Star4': 'Star'})])
# Criando um dicionário para armazenar os gêneros de cada estrela
star_genres = {}
for star in top_stars.index:
    movies = stars[stars['Star'] == star]['Genre']
    genres = set()
    for movie in movies:
        genres.update(movie.split(','))
    star_genres[star] = genres

# Contagem do número de filmes de cada gênero para cada estrela
star_genre_counts = {}
for star, genres in star_genres.items():
    genre_counts = {}
    for genre in genres:
        genre_counts[genre] = 0
    for movie in stars[stars['Star'] == star]['Genre']:
        for genre in movie.split(','):
            genre_counts[genre] += 1
    star_genre_counts[star] = genre_counts

# Criando o dataframe com os gêneros e contagens para cada estrela
df_star_genres = pd.DataFrame(star_genre_counts).transpose()

fig = px.bar(df_star_genres,
             title='Gêneros mais produzidos pelos Top 10 estrelas com maior faturamento',
             labels={'index': 'Estrela', 'value': 'Quantidade de filmes'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()


####Análises dos Filmes

In [81]:
film_gross = df.groupby('Series_Title')['Gross'].max().sort_values(ascending=False)

top_10_film = film_gross.head(10)

fig = px.bar(top_10_film, x=top_10_film.index, y='Gross',
             title='Filmes com maior faturamento',
             labels={'Series_Title': 'Filme', 'Gross': 'Faturamento (USD)'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [82]:
film_rating = df.groupby('Series_Title')['IMDB_Rating'].mean().sort_values(ascending=False)

top_10_film_rating = film_rating.head(10)

fig = px.bar(top_10_film_rating, x=top_10_film_rating.index, y='IMDB_Rating',
             title='Filmes com maiores notas no IMDB',
             labels={'Series_Title': 'Filme', 'IMDB_Rating': 'Nota no IMDB'},
             width=600,
             height=500)

fig.update_xaxes(tickangle=-45)

fig.show()

In [83]:
# prompt: Quais os generos que geram mais faturamento? me faça um gráfico usando plotly

import plotly.express as px

# Agrupar os dados por gênero e somar o faturamento
grouped_df = df.groupby('Genre')['Gross'].sum().sort_values(ascending=False)

# Criar o gráfico de barras
fig = px.bar(grouped_df, x=grouped_df.index, y='Gross',
             title='Faturamento por Gênero',
             labels={'Genre': 'Gênero', 'Gross': 'Faturamento (USD)'},
             width=600,
             height=500)

# Atualizar o eixo x para girar os rótulos
fig.update_xaxes(tickangle=-45)

# Exibir o gráfico
fig.show()


####Análises dos Faturamento

In [84]:
# Gerando boxplot para buscar Outliers
fig = px.box(df, y="Gross", title="Boxplot de Faturamento")

fig.show()

In [85]:
# Faturamento ao decorrer dos anos
grouped_df = df.groupby('Released_Year')['Gross'].sum()

fig = px.line(grouped_df, x=grouped_df.index, y='Gross',
              title='Faturamento ao decorrer dos anos',
              labels={'Released_Year': 'Ano', 'Gross': 'Faturamento (USD)'},
              width=600,
              height=500)

fig.show()


#Respondendos as perguntas

##Qual filme você recomendaria para uma pessoa que você não conhece?

In [52]:
df.sort_values('IMDB_Rating', ascending=False).head(1)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0


Eu recomendaria o filme "O poderoso Chefão", pois ele é o filme que apresenta maior nota do IMDB.

##Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme?

In [53]:
# Criar matriz de correlação
corr_matrix = df[numerical_cols].corr()

fig = px.imshow(corr_matrix, text_auto=True)

fig.update_layout(
    title="Heatmap das correlações",
    xaxis_title="Variáveis",
    yaxis_title="Variáveis",
    width=600,
    height=500
)

fig.show()


Os principais fatores para uma alta expectativa de faturamento de um filme são principalmente o elenco e a direção do filme, o seu gênero, as franquias e sequências e as tendências sociais e culturais.

##Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?


Pela coluna Overview, que é a coluna que contém uma sinopse do filme, pode ser feita uma análise de sentimento, inferir o gênero do filme e identificar temas específicos do filme. Realizando análises de processamento de linguagem natural nela podemos também verificar se

#Previsão da Nota do IMDB

3. Para prever a nota do IMDB, removi a coluna "Overview" porque continha informações específicas de um único filme, o que não ajudaria positivamente no modelo de ML que estou desenvolvendo. Após limpar e transformar os dados, excluindo colunas desnecessárias e tratando valores nulos, ajustei variáveis numéricas como pontuação no Metascore, número de votos e faturamento para floats, converti a coluna "Runtime" para números inteiros e separei as variáveis em numéricas e categóricas. Estamos resolvendo um problema de regressão para prever a nota do IMDB, e utilizei métricas como MAE, MSE e RMSE para avaliar o desempenho do modelo. O Random Forest Regression foi escolhido devido ao seu desempenho superior em relação aos outros modelos de regressão, por suas métricas e por sua robustez contra overfitting e capacidade de generalização, resultando em previsões mais precisas para a nota do IMDB.

##ML

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [93]:
# Selecionando as colunas relevantes
x = df[['Released_Year', 'Certificate', 'Runtime', 'Genre', 'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']]
y = df['IMDB_Rating']

# Codificar variáveis categóricas
categorical_cols = ['Certificate', 'Genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']
numerical_cols = ['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross']

# Pré-processamento utilizando StandardScaler e OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Dividindo os dados em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

Buscando identificar o melhor modelo de regressão para a nossa base de dados, vamos comparar as métricas MSE, RMSE e MAE, sabendo que, quanto menores forem os valores, melhor é o modelo.

In [94]:
# Definindo diferentes modelos de regressão para realizar testes
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(n_estimators=100, random_state=30),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'Support Vector Regression': SVR(),
    'K-Nearest Neighbors Regression': KNeighborsRegressor(n_neighbors=6)
}

# Avaliando cada modelo
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'{name} - MSE: {mse}, RMSE: {rmse}, MAE: {mae}')

Linear Regression - MSE: 0.03612860854554152, RMSE: 0.19007527073646777, MAE: 0.15133590034942432
Ridge Regression - MSE: 0.03531271793179802, RMSE: 0.18791678459306935, MAE: 0.15020470755247223
Lasso Regression - MSE: 0.06639577363233874, RMSE: 0.2576737736602985, MAE: 0.21292847503373816
Decision Tree Regression - MSE: 0.05755244755244758, RMSE: 0.23990091194584395, MAE: 0.183916083916084
Random Forest Regression - MSE: 0.030451076923077025, RMSE: 0.1745023693910115, MAE: 0.1346573426573432
Gradient Boosting Regression - MSE: 0.031282045427422946, RMSE: 0.17686731022838265, MAE: 0.13929694845699514
Support Vector Regression - MSE: 0.046671379706977226, RMSE: 0.21603559824014473, MAE: 0.16902064807916253
K-Nearest Neighbors Regression - MSE: 0.036660839160839155, RMSE: 0.19147020436830153, MAE: 0.14883449883449887


Analisando os dados, vemos que o Random Forest Regression se destaca ligeiramente como o melhor modelo em todas as métricas. Com isso selecionaremos o modelo Random Forest Regression para o nosso Machine Learning.